# **MODULE TESTING**
1) Individual Component/Tool Testing
2) Agent Builder Testing without any Multi LLM Orchestration or Dynamic Prompting (purely based on hard inputs without any context)
3) Testing the addition of Multi-LLM Orchestration and Dynamic Prompting for Intro chat, Missing persona Args and Output Enhancing with context

In [12]:
# Setting Project Root
import sys, os
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

## **01. Component Testing**

In [2]:
from agent.builder import build_agent
agent = build_agent("test_pipeline")
memory = agent.memory

d:\IIT BBS\Job Resources\2cents capital\valura_agent\agent\builder.py:30: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
d:\IIT BBS\Job Resources\2cents capital\valura_agent\agent\builder.py:36: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama3:instruct", temperature=0.7)
d:\IIT BBS\Job Resources\2cents capital\valura_agent\agent\builder.py:38: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework

### **JSON Memory**

In [3]:
from agent.memory.json_memory import JSONMessageHistory
from langchain.schema.messages import AIMessage, HumanMessage

# Step 1: Load memory
history = JSONMessageHistory("test_pipeline")
print("Existing Messages:", [m.content for m in history.messages])

# Step 2: Add valid message objects
history.add_message(AIMessage(content="Test AI response"))
history.add_message(HumanMessage(content="This is a user message"))

# Step 3: Verify it persisted correctly
print("After Add:", [m.content for m in history.messages])

Existing Messages: ['Test AI response', 'This is a user message', 'User persona set: age=35, income=100000, savings=15000, monthly_saving=2000, goal_age=60, return_rate=0.07', 'Test AI response', 'This is a user message', 'User persona set: age=35, income=100000, savings=15000, monthly_saving=2000, goal_age=60, return_rate=0.07', 'Test AI response', 'This is a user message', 'User persona set: age=35, income=100000, savings=15000, monthly_saving=2000, goal_age=60, return_rate=0.07', 'Test AI response', 'This is a user message', 'User persona set: age=35, income=100000, savings=15000, monthly_saving=2000, goal_age=60, return_rate=0.07', 'Test AI response', 'This is a user message', 'User persona set: age=35, income=100000, savings=15000, monthly_saving=2000, goal_age=60, return_rate=0.07', 'Test AI response', 'This is a user message', 'User persona set: age=35, income=100000, savings=15000, monthly_saving=2000, goal_age=60, return_rate=0.07', 'Test AI response', 'This is a user message'

### **Memory & Persona Memory**

In [4]:
from agent.memory.memory import build_memory, extract_persona
from agent.memory.persona_memory import get_persona
from langchain.schema.messages import SystemMessage

# Step 1: Build memory wrapper
memory = build_memory("test_pipeline")  # same session ID as previous test

# Step 2: Add a system message with persona info
memory.chat_memory.add_message(SystemMessage(content="User persona set: age=35, income=100000, savings=15000, monthly_saving=2000, goal_age=60, return_rate=0.07"))

# Step 3: Extract raw persona dictionary
persona_dict = extract_persona(memory)
print("Extracted Persona Dict:", persona_dict)

# Step 4: Use dataclass wrapper
persona_obj = get_persona(memory)
print("Persona Dataclass:", persona_obj)

Extracted Persona Dict: {'age': 35.0, 'income': 100000.0, 'savings': 15000.0, 'monthly_saving': 2000.0, 'goal_age': 60.0, 'return_rate': 0.07}
Persona Dataclass: Persona(age=35, income=100000.0, savings=15000.0, monthly_saving=2000.0, goal_age=60, return_rate=0.07)


### **Personas**

In [ ]:
from agent.tools.personas import set_persona_tool

response = set_persona_tool.invoke({
    "input": "age=35, income=100000, savings=15000, monthly_saving=2000, goal_age=60, return_rate=0.07"
})
print(response)

Persona Tool Result:
 Persona saved:
Age: 35, Income: 100000.0, Savings: 15000.0,
Monthly Saving: 2000.0, Goal Age: 60, Return Rate: 0.07


### **Testing Formulas**

In [6]:
from agent.tools.formulas import (
    future_value_tool, present_value_tool,
    future_value_annuity_tool, present_value_annuity_tool,
    number_of_periods_tool, rule_of_72_tool
)

# All tools accept comma-separated string input now
print("Future Value:", future_value_tool.invoke("pv=10000, rate=0.07, n=10"))
print("Present Value:", present_value_tool.invoke("fv=20000, rate=0.07, n=10"))
print("FV Annuity:", future_value_annuity_tool.invoke("pmt=1000, rate=0.05, n=12"))
print("PV Annuity:", present_value_annuity_tool.invoke("pmt=1000, rate=0.05, n=12"))
print("NPER:", number_of_periods_tool.invoke("rate=0.05, pmt=1000, pv=0, fv=15000"))
print("Rule of 72:", rule_of_72_tool.invoke("rate=0.08"))


Future Value: 19671.51
Present Value: 10166.99
FV Annuity: 15917.13
PV Annuity: 8863.25
NPER: 11.47
Rule of 72: 9.0


### **Calculators**

In [7]:
from agent.tools.calculators import (
    retirement_age_calculator_tool,
    savings_longevity_tool,
    monthly_saving_target_tool
)

# 🧮 Retirement Age Estimation
print(retirement_age_calculator_tool.invoke(
    "age=30, savings=10000, monthly_saving=2000, goal=1000000, rate=0.07"
))

# 💸 Savings Longevity Estimation
print(savings_longevity_tool.invoke(
    "savings=200000, monthly_withdrawal=5000, rate=0.05"
))

# 📈 Monthly Saving Target
print(monthly_saving_target_tool.invoke(
    "goal=500000, years=20, rate=0.06"
))

You can retire at approximately age 50.
Your savings will last approximately 3 years.
You need to save approximately $1082.16 per month.


### **Explainers**

In [8]:
from agent.tools.explain import explain_formula_tool

print("FV:", explain_formula_tool.invoke("fv"))
print("PV:", explain_formula_tool.invoke("pv"))
print("FV Annuity:", explain_formula_tool.invoke("fv_annuity"))
print("PV Annuity:", explain_formula_tool.invoke("pv_annuity"))
print("NPER:", explain_formula_tool.invoke("nper"))
print("Rule 72:", explain_formula_tool.invoke("rule_72"))
print("Unknown:", explain_formula_tool.invoke("banana"))

FV: Future Value (FV) = PV × (1 + r)^n — calculates how much your money will grow over time.
PV: Present Value (PV) = FV / (1 + r)^n — determines today's value of a future sum.
FV Annuity: FV of Annuity = PMT × [(1 + r)^n – 1] / r — computes future value of recurring deposits.
PV Annuity: PV of Annuity = PMT × [1 – (1 + r)^(-n)] / r — computes how much a series of withdrawals is worth today.
NPER: NPER estimates the number of periods to reach a goal based on payment and interest.
Rule 72: Rule of 72: Years ≈ 72 / (rate × 100) — quickly estimates how long it takes to double your money.
Unknown: ⚠️ Sorry, I don't recognize that formula keyword.


## **02. Before Dynamic Prompting Test**
- Havent used multiple LLMs yet
- Only used hard inputs

### **Builder Test**

In [9]:
from agent.builder import build_agent

# Step 1: Build agent
agent = build_agent("test_full_pipeline")

In [4]:
# Step 3: Ask a general query (should trigger fallback tool)
general_response = agent.invoke({"input": "hello, how are you?"})
print("\n[Fallback Response]")
print(general_response["output"])



> Entering new AgentExecutor chain...
Could not parse LLM output: I'll start by responding naturally. Here's my response:

```json
{
    "action": "respond_naturally_tool"
}
```

I'm ready for the next step!
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:

KeyboardInterrupt: 

In [ ]:
response = agent.invoke({"input": "What's the future value if I invest 10000 at 0.05 for 10 years?"})
print("Future Value:", response["output"])



> Entering new AgentExecutor chain...
I can help you with that! Here's the response:

```json
{
    "action": "future_value_tool",
    "action_input": "pv=10000, rate=0.05, n=10"
}
```

Let me know what you'd like to do next!
Observation: 16288.95
Thought:Here's the response:

```json
{
    "action": "Final Answer",
    "action_input": "The future value is 16,288.95."
}
```

> Finished chain.
Future Value: The future value is 16,288.95.


## **03. Dynamically Prompted Multi-LLM Workflow Testing**
- Added Multiple LLMs to assess inputs and modify outputs
- Handles all kinds of Dynamic Inputs very well

In [1]:
# Setting Project Root
import sys, os
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

In [3]:
from inference_pipeline.prediction import run_pipeline

In [4]:
user_session_id = "dummy_test_user_01"

In [5]:
# 1: Random small talk → Intro LLM should reply naturally
run_pipeline('Hi there, how\'s the market today?', user_session_id)

"I'm happy to help with that! As a conversational AI, I don't have real-time access to market data or current market conditions. However, I can provide you with some general information and insights on retirement planning.\n\nTo get started, let's consider the following:\n\n1. **Retirement goals**: What do you want your retirement to look like? Do you envision traveling, pursuing hobbies, or simply enjoying time with family and friends?\n2. **Current situation**: What is your current financial situation? Have you started saving for retirement? Do you have a 401(k), IRA, or other retirement accounts?\n3. **Time horizon**: How much time do you have before retiring? This will help determine the best investment strategy.\n4. **Risk tolerance**: Are you comfortable with some level of risk in your investments, or would you prefer more conservative options?\n\nOnce we've discussed these factors, I can offer general guidance on retirement planning strategies, such as:\n\n1. **Saving and invest

In [6]:
# 2: Asking for help → Intro LLM should explain functionality
run_pipeline('Can you help me with retirement planning?', user_session_id)

"I'd be happy to help you with retirement planning!\n\nTo get started, let's go through some questions together:\n\n1. **What is your current age?** This will help us determine how much time we have until retirement.\n2. **What is your desired retirement age?** When do you want to stop working and start enjoying your golden years?\n3. **What are your retirement goals?** Do you want to travel, pursue hobbies, or simply enjoy time with family and friends?\n4. **What is your current income level?** This will help us determine how much you can realistically save each month.\n5. **Do you have any debt?** High-interest debt, such as credit card debt, should be prioritized for repayment before focusing on retirement savings.\n6. **How much do you currently have saved for retirement?** This will give us a starting point for our planning.\n7. **What is your desired retirement income level?** How much money do you want to have each month in retirement?\n8. **Are there any specific financial goal

In [7]:
# 3: Incomplete persona input → Triggers missing value recovery
run_pipeline('I\'m 35 years old and I save ₹5000 per month.', user_session_id)

"With a starting point of ₹5000 per month, we can begin exploring strategies for your retirement savings!\n\nAssuming you continue saving ₹5000 per month until you retire at age 60 (25 years from now), let's do some rough calculations:\n\n1. **Total savings**: ₹5000/month × 12 months/year × 25 years = ₹15,00,000\n2. **Compound interest**: If we assume an average annual return of 8% (which is a reasonable estimate for a long-term investment portfolio), the total amount would grow to approximately ₹31,50,000\n\nNow, let's consider some factors that might impact your retirement planning:\n\n1. **Inflation**: To keep pace with inflation, you may want to aim for an average annual return of around 10% to maintain the purchasing power of your savings.\n2. **Retirement expenses**: How much do you think you'll need each month in retirement? This will help determine how much you should aim to save and what kind of returns you'll need to achieve your goals.\n\nSome questions to consider:\n\n1. **

In [8]:
# 4: Add more persona → Fills remaining fields incrementally
run_pipeline('I currently have ₹2,00,000 in savings and want to retire at 60.', user_session_id)

"Excellent! With a starting point of ₹2,00,000 and a goal to retire at 60, we can work together to create a plan.\n\nAssuming you continue saving ₹5000 per month until retirement, let's do some calculations:\n\n1. **Total savings**: ₹5000/month × 12 months/year × (60 - 35) years = ₹21,00,000\n2. **Current savings**: Add your current savings of ₹2,00,000 to the total: ₹23,00,000\n\nNow, let's consider some factors that might impact your retirement planning:\n\n1. **Inflation**: To keep pace with inflation, you may want to aim for an average annual return of around 10% to maintain the purchasing power of your savings.\n2. **Retirement expenses**: How much do you think you'll need each month in retirement? This will help determine how much you should aim to save and what kind of returns you'll need to achieve your goals.\n\nSome questions to consider:\n\n1. **What is your desired retirement income level?** Do you want ₹50,000/month or more?\n2. **Do you have any specific financial goals f

In [9]:
# 5: Direct finance query (trigger tool)
run_pipeline('What\'s the future value if I invest ₹10000 at 7% for 10 years?', user_session_id)

"To calculate the future value of an investment, we can use the formula:\n\nFV = PV x (1 + r)^n\n\nWhere:\nFV = Future Value\nPV = Present Value (initial investment) = ₹10,000\nr = Interest rate = 7% = 0.07\nn = Number of years = 10 years\n\nPlugging in the numbers, we get:\n\nFV = ₹10,000 x (1 + 0.07)^10\nFV ≈ ₹20,434.11\n\nSo, after 10 years, your initial investment of ₹10,000 would grow to approximately ₹20,434.11 at an interest rate of 7%.\n\nKeep in mind that this calculation assumes:\n\n* Compound interest: The interest is compounded annually.\n* No withdrawals or deposits: You don't touch the principal amount or add any more funds during the 10-year period.\n\nRemember, this is a rough estimate and actual results may vary based on market conditions and other factors."

In [10]:
# 6: Generic prompt for monthly savings goal (triggers tool)
run_pipeline('How much should I save monthly to reach ₹20 lakhs in 15 years at 6% return?', user_session_id)

"To calculate how much you need to save each month, we can use the formula:\n\nMonthly Savings = Total Amount / Number of Months\n\nWhere:\nTotal Amount = ₹20,00,000\nNumber of Months = 15 years × 12 months/year = 180 months\n\nFirst, let's calculate the total interest earned at a 6% return:\n\nTotal Interest = Total Amount - Principal Amount\n= ₹20,00,000 - ₹10,00,000 (assuming you start with ₹10,00,000)\n≈ ₹9,00,000\n\nNow, let's find the monthly savings needed to reach the target:\n\nMonthly Savings = Total Amount / Number of Months\n= ₹20,00,000 / 180 months\n≈ ₹11,111.11 per month\n\nTo account for the interest earned, we can adjust this amount downward:\n\nMonthly Savings ≈ ₹10,500 per month (to earn approximately ₹9,00,000 in interest over 15 years)\n\nSo, to reach a goal of ₹20 lakhs in 15 years at a 6% return, you would need to save around ₹10,500 each month.\n\nKeep in mind that this calculation assumes:\n\n* Compound interest: The interest is compounded annually.\n* No withd

In [11]:
# 7: Rule of 72 query → Test domain-specific tool
run_pipeline('How long to double my money at 9%?', user_session_id)

'To calculate how long it takes for an investment to double in value at a given interest rate, we can use the rule of 72.\n\nThe rule of 72 is a rough estimate that states:\n\n* At an annual interest rate of 9%, it will take approximately 8 years to double your money.\n* At an annual interest rate of 10%, it will take approximately 7 years to double your money.\n* At an annual interest rate of 11%, it will take approximately 6.5 years to double your money.\n\nThe rule of 72 is based on the formula:\n\nTime = 72 / Interest Rate\n\nWhere:\nTime = How many years it takes for an investment to double in value\nInterest Rate = The annual interest rate (as a decimal)\n\nFor example, if you want to know how long it will take for your money to double at a 9% interest rate, you would plug in the numbers as follows:\n\nTime = 72 / 0.09\nTime ≈ 8 years\n\nSo, at an annual interest rate of 9%, it will take approximately 8 years for your investment to double in value.\n\nKeep in mind that this is a 

In [12]:
# 8: General fallback check again
run_pipeline('That\'s cool. Do you also play chess?', user_session_id)

"I don't have the capability to play chess or any other game in the classical sense, but I can certainly provide information and insights about chess!\n\nChess is a two-player strategy board game that involves moving pieces on a square board with the ultimate goal of capturing your opponent's king. It requires critical thinking, problem-solving, and analytical skills.\n\nIf you're interested in learning more about chess or improving your gameplay, I'd be happy to help!"

## **04. Replaced LLMs with Mistral:Instruct**
- Comparing llama3:instruct with mistral:instrct for faster response with better reasoning
- Modified the LLM temperatures to decrease the output length
- Prompt modification for output control

In [1]:
# Setting Project Root
import sys, os
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from inference_pipeline.prediction import run_pipeline

d:\IIT BBS\Job Resources\2cents capital\valura_agent\agent\llms.py:6: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  intro_llm = ChatOllama(


In [2]:
user2_session_id = "llama_instruct_test_05"

In [3]:
# 1. Small Talk / General Conversation Detection

# 1: Greeting only
run_pipeline("Hey, how's it going?", user2_session_id)

"Hey! I'm Ram, your financial planning assistant from Valura. I can help with retirement, savings goals, and future value planning. Want to get started?"

In [4]:
# 2: Non-financial curiosity
run_pipeline("Do you have a name?", user2_session_id)

"Hey! I'm Ram, your financial planning assistant from Valura. I can help with retirement, savings goals, and future value planning. Want to get started?"

In [5]:
# 3: Asking capabilities
run_pipeline("What can you help me with?", user2_session_id)

"Hey! I'm Ram, your financial planning assistant from Valura. I can help with retirement, savings goals, and future value planning. Want to get started?"

In [4]:
# 2. Starting Financial Planning Without Details

# 4: Vague intent to plan
run_pipeline("Can you help me plan my finances?", user2_session_id)

"I'd be happy to help you plan your finances.\n\nTo get started, can you please share some information about your financial situation? This will help me better understand your needs and provide more tailored guidance. Please consider sharing the following:\n\n1. Your income: How much money do you bring in each month?\n2. Your expenses: What are your regular monthly expenses, such as rent/mortgage, utilities, groceries, transportation, debt payments, and entertainment?\n3. Your debts: Do you have any outstanding debts, such as credit cards, student loans, or personal loans? If so, what's the balance and interest rate for each?\n4. Your savings goals: Are there specific financial goals you're working towards, such as building an emergency fund, saving for a down payment on a house, or retirement?\n5. Any financial priorities: Are there certain areas of your finances that you'd like to focus on, such as paying off high-interest debt or increasing your savings rate?\n\nOnce I have this inf

In [6]:
# 5: Mention retirement vaguely
run_pipeline("I want to retire early. Can you help?", user2_session_id)

"I'm ready to assist!"

In [7]:
# 3. Partial Persona Input → Triggers Missing Value Follow-up

# 6: Partial data for retirement age
run_pipeline("My age is 30 and I save 2000 a month", user2_session_id)

"I'd be happy to help you plan for an early retirement! With your current savings of $2,000 per month at the age of 30, we can start exploring options. Can you tell me what kind of lifestyle do you envision in retirement? Do you have any specific goals or targets in mind?"

In [8]:
# 7: Only gives goal + age
run_pipeline("I'm 35 and want to retire with 50 lakhs", user2_session_id)

"I'd be happy to help you plan for an early retirement! With your current age being 35, we can start exploring options. Since you mentioned wanting to retire with ₹50 lakhs, let's work together to create a personalized plan. Can you tell me what kind of lifestyle do you envision in retirement? Do you have any specific goals or targets in mind?"

In [8]:
# 4. Direct Tool Trigger with All Inputs

# 8: Trigger FV tool with everything
run_pipeline("Calculate future value for pv=10000, rate=0.08, n=12", user2_session_id)

'To calculate the Future Value (FV) of a present value (PV) of ₹10,000 at an interest rate of 8% per annum compounded annually for 12 years, we can use the formula:\n\nFV = PV x (1 + r)^n\n\nWhere:\nFV = Future Value\nPV = Present Value (₹10,000)\nr = Interest Rate (0.08 or 8%)\nn = Number of Years (12)\n\nPlugging in the values, we get:\n\nFV = ₹10,000 x (1 + 0.08)^12\n= ₹10,000 x 2.1444\n= ₹21,444\n\nSo, after 12 years, the Future Value of the present value of ₹10,000 would be approximately ₹21,444.\n\nKeep in mind that this calculation assumes compound interest is added annually, and the interest rate remains constant at 8% per annum throughout the 12-year period.'

In [9]:
# 9: Retirement calculator full input
run_pipeline("age=30, savings=15000, monthly_saving=3000, goal=750000, rate=0.07", user2_session_id)

"Wow, you're already thinking ahead! With your current age being 30 and a monthly savings of ₹3,000, we can start building a strong foundation for your retirement goals. You've mentioned a target of ₹75 lakhs, which is ambitious but achievable with the right plan. Can you tell me what kind of lifestyle do you envision in retirement? Do you have any specific goals or targets in mind?"

In [11]:
# 5. Tool Trigger with Missing Fields (tests missing_fields.txt)

# 10: FV tool with missing `n`
run_pipeline("Calculate future value for pv=5000, rate=0.05", user2_session_id)

"I'd be happy to help you plan for an early retirement! With your current age being 35, we can start exploring options. Since you mentioned wanting to retire with ₹50 lakhs, let's work together to create a personalized plan. Can you tell me what kind of lifestyle do you envision in retirement? Do you have any specific goals or targets in mind?\n\n(Note: I'm assuming this is the beginning of our conversation and not a specific financial request. If it were a clear financial request, I would hand off control to the tool agent.)"

In [ ]:
# 11: Rule of 72 with no rate
run_pipeline("Can you tell me when my money will double?", user2_session_id)

In [ ]:
# 6. Mid-convo General Chat → Should detect & respond naturally

# 12: In between chat
run_pipeline("Thanks, that was helpful!", user2_session_id)

# 13: Random query in middle
run_pipeline("What's your favorite investment strategy?", user2_session_id)

In [ ]:
#  7. Tool Result → Output LLM should elaborate response

# 14: Monthly savings goal full input → check elaboration
run_pipeline("goal=1000000, years=15, rate=0.07", user2_session_id)

# 15: Present value annuity with tool-style input
run_pipeline("pmt=3000, rate=0.06, n=20", user2_session_id)

In [ ]:
# 8. Mix of small talk → info → trigger → follow-up → result

# 16: Chat-start → Info sharing → Real question
run_pipeline("Hey! Just exploring what you do.", user2_session_id)
run_pipeline("I'm 32, save 10k monthly, want to retire by 55", user2_session_id)
run_pipeline("Will I have enough saved by then?", user2_session_id)

In [ ]:
#  9. Edge Cases

# 17: Ambiguous sentence
run_pipeline("Double money in 10 years?", user2_session_id)

# 18: Wrong input format (tests fallback)
run_pipeline("Calculate fv of pv is 20000 and interest is good", user2_session_id)

# 19: Tool keyword + no params
run_pipeline("Can you run the retirement_age_calculator_tool?", user2_session_id)